In [24]:
import sympy as sp
import numpy as np

sp.init_printing(use_latex=True)

### Rotation convention
The yaw, pitch, roll (YPR) convention common in aerospace follows an intrinsic (that is, about the moving coordinate system) rotation about the body Z, Y, and X axes. 

For the calculation of the associated rotation matrices see the code below. This also shows that the YPR convention is equal to an extrinsic (that is, about the inertial coordinate system) rotation in the order XYZ.

The convention used here thus corresponds to the Euler ZYX mentioned in this source: http://web.mit.edu/2.05/www/Handout/HO2.PDF \
And this corresponds to the rotation matrix derived here: https://link.springer.com/article/10.1007/s11071-022-08212-w

In [25]:
L1_length = 0.110
L2_length = 0.311
L3_length = 0.273

#q1, q2, q3 = sp.symbols('q_1 q_2 q_3', real=True)
x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3 = sp.symbols('x_b y_b z_b yaw pitch roll q_1 q_2 q_3', real=True)
state_vector = sp.Matrix([x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3])

# Secondary states
x_be, y_be, z_be = sp.symbols('x^{b}_{be} y^{b}_{be} z^{b}_{be}')

# Parameters
L1, L2, L3 = sp.symbols('L_1 L_2 L_3', real=True, positive=True)

In [26]:
# ROTATIONS
# Intrinsic ZYX
Rz = sp.Matrix([
    [sp.cos(yaw), -sp.sin(yaw), 0],
    [sp.sin(yaw),  sp.cos(yaw), 0],
    [0,            0,           1]
])

Ry = sp.Matrix([
    [sp.cos(pitch),  0, sp.sin(pitch)],
    [0,              1, 0            ],
    [-sp.sin(pitch), 0, sp.cos(pitch)]
])

Rx = sp.Matrix([
    [1, 0,            0           ],
    [0, sp.cos(roll), -sp.sin(roll)],
    [0, sp.sin(roll),  sp.cos(roll)]
])

# Intrinsic ZYX: Postmultiplication
R_zyx_intrinsic = Rz * Ry * Rx

# Extrinsic XYZ: Premultiplication
R_xyz_extrinsic = Rz * Ry * Rx

sp.pprint(R_xyz_extrinsic)
sp.pprint(R_zyx_intrinsic)
R_b = R_zyx_intrinsic

⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)                     sin(roll)⋅cos(pitch)                      ↪

↪ pitch)⋅cos(roll)⋅cos(yaw) + sin(roll)⋅sin(yaw)⎤
↪                                               ⎥
↪ pitch)⋅sin(yaw)⋅cos(roll) - sin(roll)⋅cos(yaw)⎥
↪                                               ⎥
↪            cos(pitch)⋅cos(roll)               ⎦
⎡cos(pitch)⋅cos(yaw)  sin(pitch)⋅sin(roll)⋅cos(yaw) - sin(yaw)⋅cos(roll)  sin( ↪
⎢                                                                              ↪
⎢sin(yaw)⋅cos(pitch)  sin(pitch)⋅sin(roll)⋅sin(yaw) + cos(roll)⋅cos(yaw)  sin( ↪
⎢                                                                              ↪
⎣    -sin(pitch)    

### Angular velocity
Taking the time-derivative of expressions of Euler angles does not yield angular velocities. A transformation matrix exists to convert between them. For the YPR convention described above, this looks like:
$$
T(\phi_{b})=\begin{bmatrix} 0 & -\sin(\psi) & \cos(\psi) \cos(\theta) \\
                            0 & \cos(\psi) & \sin(\psi) \cos(\theta) \\
                            1 & 0 & 0 \end{bmatrix}
$$

Source: https://ieeexplore.ieee.org/document/6608869

In [27]:
T_euler = sp.Matrix([[0, -sp.sin(yaw), sp.cos(yaw)*sp.cos(pitch)],
                     [0, sp.cos(yaw), sp.sin(yaw)*sp.cos(pitch)],
                     [1, 0, -sp.sin(pitch)]])

T_A = sp.Matrix([[sp.eye(3), sp.zeros(3, 3)],
                 [sp.zeros(3, 3), T_euler]])

### Conventions
For arm 1, the nominal configuration is straight out to the right of the body (i.e. along positive $Y_{b}$). For arm 2, the nominal configuration is straight out to the left of the body (i.e. along the negative $Y_{b}$).

Rotations around relative axes are intrinsic and thus you use postmultiplication of the rotation.

In [28]:
# Body frame b: FRD at vehicle CoM
T_Ib = sp.Matrix([[sp.cos(yaw)*sp.cos(pitch), sp.cos(yaw)*sp.sin(pitch)*sp.sin(roll)-sp.sin(yaw)*sp.cos(roll), sp.cos(yaw)*sp.sin(pitch)*sp.cos(roll)+sp.sin(yaw)*sp.sin(roll), x_b],
                  [sp.sin(yaw)*sp.cos(pitch), sp.sin(yaw)*sp.sin(pitch)*sp.sin(roll)+sp.cos(yaw)*sp.cos(roll), sp.sin(yaw)*sp.sin(pitch)*sp.cos(roll)-sp.cos(yaw)*sp.sin(roll), y_b],
                  [-sp.sin(pitch), sp.cos(pitch)*sp.sin(roll), sp.cos(pitch)*sp.cos(roll), z_b],
                  [0, 0, 0, 1]])

# Frame 0: Z axis of rotation pointing forward, X axis pointing up, Y axis pointing right (90 deg rotation around Yb)
# Rotation is to 0 from b
T_b0 = sp.Matrix([[0, 0, 1, 0],
                  [0, 1, 0, 0],
                  [-1, 0, 0, 0],
                  [0, 0, 0, 1]])

# Frame 1: Rotate from 0 around z0 by q1, then rotate around y0 by -pi/2
rot_z0 = sp.Matrix([[sp.cos(q1), -sp.sin(q1), 0],
                    [sp.sin(q1), sp.cos(q1), 0],
                    [0, 0, 1]])
rot_y0 = sp.Matrix([[sp.cos(-sp.pi/2), 0, sp.sin(-sp.pi/2)],
                    [0, 1, 0],
                    [-sp.sin(-sp.pi/2), 0, sp.cos(-sp.pi/2)]])
rot_z0y0 = rot_z0 * rot_y0 # postmultiply because y0 is relative axis
T_01 = sp.diag(rot_z0y0, 1)
T_01[1,3] = L1 # Location of 1 in frame 0
print("Transformation matrix T_01")
sp.pprint(T_01)

# Frame 2
rot_y1 = sp.Matrix([[sp.cos(sp.pi/2), 0, sp.sin(sp.pi/2)],
                    [0, 1, 0],
                    [-sp.sin(sp.pi/2), 0, sp.cos(sp.pi/2)]])
rot_z1 = sp.Matrix([[sp.cos(q2), -sp.sin(q2), 0],
                    [sp.sin(q2), sp.cos(q2), 0],
                    [0, 0, 1]])
rot_z1y1 = rot_z1 * rot_y1 # postmultiply because z2 is relative axis
T_12 = sp.diag(rot_z1y1, 1)
T_12[1, 3] = L2 # Location of 2 in frame 1
print("Transformation matrix T_12")
sp.pprint(T_12)

# Frame 3
rot_z2 = sp.Matrix([[sp.cos(sp.pi/2), -sp.sin(sp.pi/2), 0],
                    [sp.sin(sp.pi/2), sp.cos(sp.pi/2), 0],
                    [0, 0, 1]])
rot_z2_prime = sp.Matrix([[sp.cos(q3), -sp.sin(q3), 0],
                    [sp.sin(q3), sp.cos(q3), 0],
                    [0, 0, 1]])
rot_z2y2 = rot_z2 * rot_z2_prime
T_23 = sp.diag(rot_z2y2, 1)
T_23[1, 3] = L3
print("Transformation matrix T_23")
sp.pprint(T_23)

# Frame e: Translation along Y3 by L3, rotation about X3 by -pi/2
# This yields the final frame e with positive Z pointing out, X pointing up and Y completing
T_3e = sp.Matrix([[ 0,  0, 1, 0],
                  [-1,  0, 0, 0],
                  [ 0, -1, 0, 0],
                  [ 0,  0, 0, 1]])

T_be = sp.trigsimp(sp.expand(T_b0*T_01*T_12*T_23*T_3e))
    
# Set parameters
T_be = T_be.subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])

Transformation matrix T_01
⎡0  -sin(q₁)  -cos(q₁)  0 ⎤
⎢                         ⎥
⎢0  cos(q₁)   -sin(q₁)  L₁⎥
⎢                         ⎥
⎢1     0         0      0 ⎥
⎢                         ⎥
⎣0     0         0      1 ⎦
Transformation matrix T_12
⎡0   -sin(q₂)  cos(q₂)  0 ⎤
⎢                         ⎥
⎢0   cos(q₂)   sin(q₂)  L₂⎥
⎢                         ⎥
⎢-1     0         0     0 ⎥
⎢                         ⎥
⎣0      0         0     1 ⎦
Transformation matrix T_23
⎡-sin(q₃)  -cos(q₃)  0  0 ⎤
⎢                         ⎥
⎢cos(q₃)   -sin(q₃)  0  L₃⎥
⎢                         ⎥
⎢   0         0      1  0 ⎥
⎢                         ⎥
⎣   0         0      0  1 ⎦


In [29]:
# Denavit-Hartenberg approach
# Parameters
DH_theta = [q1, q2, q3]
DH_alpha = [-sp.pi/2, sp.pi/2, 0]
DH_r = [L1, L2, L3]
DH_d = [0, 0, 0]

# Frame 0: X pointing right, Z pointing forward, Y pointing down (i.e. rotation around Zb by pi/2, then rotation around Xb' by pi/2)
# I.e. intrinisic rotation so multiply Rz * Rx'
# Frame 1: Attached at Q2, Z pointing down, X pointing right (rotation around X0 of -pi/2, then translation along x0' by L1)

H_01 = sp.Matrix([[sp.cos(DH_theta[0]), -sp.sin(DH_theta[0])*sp.cos(DH_alpha[0]), sp.sin(DH_theta[0])*sp.sin(DH_alpha[0]), DH_r[0]*sp.cos(DH_theta[0])],
                  [sp.sin(DH_theta[0]), sp.cos(DH_theta[0])*sp.cos(DH_alpha[0]), -sp.cos(DH_theta[0])*sp.sin(DH_alpha[0]), DH_r[0]*sp.sin(DH_theta[0])],
                  [0, sp.sin(DH_alpha[0]), sp.cos(DH_alpha[0]), DH_d[0]],
                  [0, 0, 0, 1]])

# Frame 2: Attached at Q3, Z pointing forward, X pointing right (rotation around X1 of pi/2, then translation along x1' by L2)
H_12 = sp.Matrix([[sp.cos(DH_theta[1]), -sp.sin(DH_theta[1])*sp.cos(DH_alpha[1]), sp.sin(DH_theta[1])*sp.sin(DH_alpha[1]), DH_r[1]*sp.cos(DH_theta[1])],
                  [sp.sin(DH_theta[1]), sp.cos(DH_theta[1])*sp.cos(DH_alpha[1]), -sp.cos(DH_theta[1])*sp.sin(DH_alpha[1]), DH_r[1]*sp.sin(DH_theta[1])],
                  [0, sp.sin(DH_alpha[1]), sp.cos(DH_alpha[1]), DH_d[1]],
                  [0, 0, 0, 1]])

# Frame 3: Attached at EE, Z pointing forward, X pointing right (translation along x2 by L3)
H_23 = sp.Matrix([[sp.cos(DH_theta[2]), -sp.sin(DH_theta[2])*sp.cos(DH_alpha[2]), sp.sin(DH_theta[2])*sp.sin(DH_alpha[2]), DH_r[2]*sp.cos(DH_theta[2])],
                  [sp.sin(DH_theta[2]), sp.cos(DH_theta[2])*sp.cos(DH_alpha[2]), -sp.cos(DH_theta[2])*sp.sin(DH_alpha[2]), DH_r[2]*sp.sin(DH_theta[2])],
                  [0, sp.sin(DH_alpha[2]), sp.cos(DH_alpha[2]), DH_d[2]],
                  [0, 0, 0, 1]])

# Transformation between b and 0
R_b0 = sp.Matrix([[sp.cos(sp.pi/2), -sp.sin(sp.pi/2), 0], # Rz
                  [sp.sin(sp.pi/2), sp.cos(sp.pi/2), 0],
                  [0, 0, 1]]) * \
        sp.Matrix([[1, 0, 0],
                  [0, sp.cos(sp.pi/2), -sp.sin(sp.pi/2)], # Rx
                  [0, sp.sin(sp.pi/2), sp.cos(sp.pi/2)]])                    
assert R_b0 == sp.Matrix([[0, 0, 1], [1, 0, 0], [0, 1, 0]])
H_b0 = sp.diag(R_b0, 1)

# Transformation between 3 and e - To align e with frame e from direct derivation
R_3e = sp.Matrix([[sp.cos(sp.pi/2), 0, sp.sin(sp.pi/2)], # Ry by pi/2
                 [0, 1, 0],
                 [-sp.sin(sp.pi/2), 0, sp.cos(sp.pi/2)]]) * \
        sp.Matrix([[sp.cos(-sp.pi/2), -sp.sin(-sp.pi/2), 0],  # Rz by -pi/2
                  [sp.sin(-sp.pi/2), sp.cos(-sp.pi/2), 0],
                  [0, 0, 1]])
assert R_3e == sp.Matrix([[0, 0, 1], [-1, 0, 0], [0, -1, 0]])
H_3e = sp.diag(R_3e, 1)
H_be = (H_b0 * H_01 * H_12 * H_23 * H_3e).subs([(L1, L1_length), (L2, L2_length), (L3, L3_length)])

In [33]:
# Testing if the DH formulation yielded the same result as the direct derivation
if not ( H_be == T_be):
    #sp.pprint(H_be)
    #sp.pprint(T_be)
    # Test with random inputs
    q1_val = np.random.rand()
    q2_val = np.random.rand()
    q3_val = np.random.rand()
    x_b_val = np.random.rand()
    y_b_val = np.random.rand()
    z_b_val = np.random.rand()
    yaw_val = np.random.rand()
    pitch_val = np.random.rand()
    roll_val = np.random.rand()
    H_be_subs = H_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val), (x_b, x_b_val), (y_b, y_b_val), (z_b, z_b_val), (yaw, yaw_val), (pitch, pitch_val), (roll, roll_val)])
    T_be_subs = T_be.subs([(q1, q1_val), (q2, q2_val), (q3, q3_val), (x_b, x_b_val), (y_b, y_b_val), (z_b, z_b_val), (yaw, yaw_val), (pitch, pitch_val), (roll, roll_val)])
    sp.pprint(H_be_subs)
    sp.pprint(T_be_subs)
    
else:
    print("Kinematics verified")

⎡-0.0337375580158097   -0.99656227301371   -0.0756664601053719  -0.04642238049 ↪
⎢                                                                              ↪
⎢ 0.967086256256125   -0.0516570104771918   0.249150007483589    0.32985430636 ↪
⎢                                                                              ↪
⎢-0.252202200901664   -0.0647702807953405   0.96550445912282     0.59188693074 ↪
⎢                                                                              ↪
⎣         0                    0                    0                    1     ↪

↪ 83663⎤
↪      ⎥
↪ 0278 ⎥
↪      ⎥
↪ 5313 ⎥
↪      ⎥
↪      ⎦
⎡-0.0337375580158097   -0.99656227301371   -0.0756664601053719  -0.02261724845 ↪
⎢                                                                              ↪
⎢ 0.967086256256125   -0.0516570104771918   0.249150007483589    0.47355197884 ↪
⎢                                                                              ↪
⎢-0.252202200901664   -0.0647702807953405   0

In [17]:
# Forward kinematics function
FK_manipulator = sp.Matrix([T_be[0,3],
                T_be[1,3],
                T_be[2,3],
                sp.atan2(T_be[1,0], T_be[0,0]), # yaw
                sp.atan2(-T_be[2,0], sp.sqrt(T_be[0,0]**2 + T_be[1,0]**2)), # pitch
                sp.atan2(T_be[2,1], T_be[2,2])]) # roll


sp.pprint("State dependencies of manipulator kinematics")
print(f"x_bbe = f({FK_manipulator[0,0].free_symbols})")
print(f"y_bbe = f({FK_manipulator[1,0].free_symbols})")
print(f"z_bbe = f({FK_manipulator[2,0].free_symbols})")
print(f"yaw_bbe = f({FK_manipulator[3,0].free_symbols})")
print(f"pitch_bbe = f({FK_manipulator[4,0].free_symbols})")
print(f"roll_bbe = f({FK_manipulator[5,0].free_symbols})")



State dependencies of manipulator kinematics
x_bbe = f({q_2, q_3})
y_bbe = f({q_2, q_3, q_1})
z_bbe = f({q_2, q_3, q_1})
yaw_bbe = f({q_2, q_3, q_1})
pitch_bbe = f({q_2, q_3, q_1})
roll_bbe = f({q_2, q_3, q_1})


In [ ]:
# Forward kinematics function
T_Ie = T_Ib * T_be
FK_centralised = sp.Matrix([T_Ie[0,3],
                T_Ie[1,3],
                T_Ie[2,3],
                sp.atan2(T_Ie[1,0], T_Ie[0,0]), # yaw
                sp.atan2(-T_Ie[2,0], sp.sqrt(T_Ie[0,0]**2 + T_Ie[1,0]**2)), # pitch
                sp.atan2(T_Ie[2,1], T_Ie[2,2])]) # roll
FK_centralised = sp.trigsimp(sp.expand(FK_centralised))
sp.pprint("State dependencies of centralised kinematics")
print(f"xe = f({FK_centralised[0,0].free_symbols})")
print(f"ye = f({FK_centralised[1,0].free_symbols})")
print(f"ze = f({FK_centralised[2,0].free_symbols})")
print(f"yawe = f({FK_centralised[3,0].free_symbols})")
print(f"pitche = f({FK_centralised[4,0].free_symbols})")
print(f"rolle = f({FK_centralised[5,0].free_symbols})")

### Analysis of Euler angle singularity
When calculating the end-effector yaw and pitch, there is a division by zero which causes an Euler angle singularity. Therefore, we examine denominators in the expressions for $\theta_{e}$ and $\psi_{e}$. They are rather long and cumbersome expressions on their own due to the many rotations, so we just look at the states they are a function of.

We can simplify a little bit by setting both the body pitch and roll to zero, since the platform needs this for stabilization, and these values are likely to cause or contribute to the singularity. We look again at the independent states. Now, the expression is also much less cumbersome and we can investigate.

In [ ]:
# checking out denominator of expression for yawe and pitche
print("Free variables in denominator of yawe and pitche:")
sp.pprint(T_Ie[2,2].free_symbols)
sp.pprint(T_Ie[0,0].free_symbols)

# substitute zero pitch and roll since that is the nominal configuration
T_Ie_22 = T_Ie[2,2].subs([(roll, 0.0), (pitch, 0.0)])
T_Ie_00 = T_Ie[0,0].subs([(roll, 0.0), (pitch, 0.0)])
print("\nWith zero pitch and roll angle:")
print(f"T_Ie = f({T_Ie_22.free_symbols})")
print(f"T_Ie = f({T_Ie_00.free_symbols})")
sp.pprint(T_Ie_22)
sp.pprint(T_Ie_00)

Free variables in denominator of yawe and pitche:
{pitch, q₁, q₂, q₃, roll}
{pitch, q₁, q₂, q₃, roll, yaw}

With zero pitch and roll angle:
T_Ie = f({q_2, q_3, q_1})
T_Ie = f({q_2, q_3, q_1, yaw})
sin(q₁)⋅cos(q₂)⋅cos(q₃) + sin(q₃)⋅cos(q₁)
-(sin(q₁)⋅cos(q₃) + sin(q₃)⋅cos(q₁)⋅cos(q₂))⋅sin(yaw) + sin(q₂)⋅sin(q₃)⋅cos(yaw)


The two lines above mean that no singularities (expression evaluates to 0) exist purely dependent on the yaw.
For end-effector yaw, a singularity exists when $q_{1}$=0 and $q_{3}$=0 (fully stretched sideways), which is intuitive. When $\cos(q_{2})$ is 0, a singularity could exist, but this is irrelevant since the associated angle ($q_{2}$=$\pi /2$) is outside the joint limit.

For the end-effector pitch, a couple of conditions for Euler angle singularities exist. We can evaluate them by looking at when the factors in the expression evaluate to zero. With this method, the following set of singularities can be derived:
$$
\phi_{b} = 0 \land q_{2}=0 \\
\phi_{b} = 0 \land q_{3}=0 \\
q_{1}=0 \land q_{3}=0 \\
\phi_{b} = \pi /2 \land q_{1}=\pi /2 \land q_{3}=\pi /2
$$
Of these, condition IV is also not feasible because $q_{3}=\pi /2$ is outside the joint limit for $q_{3}$

### Controllability
Since the states roll and y_b, and pitch and x_b are coupled, not the entire end-effector configuration space is reachable.

In [ ]:
# Jacobian calculation for velocity
J_centralised = FK_centralised.jacobian([x_b, y_b, z_b, yaw, pitch, roll, q1, q2, q3])

print(f"Centralised Jacobian shape: {J_centralised.shape}")

J_manipulator = FK_manipulator.jacobian([q1, q2, q3])
J_b = sp.Matrix([[sp.eye(3), -(R_b*sp.Matrix([[FK_manipulator[0,0]],[FK_manipulator[1,0]],[FK_manipulator[2,0]]])).hat()],
                [sp.zeros(3,3), sp.eye(3)]])

J_eb = sp.Matrix([[R_b, sp.zeros(3,3)],[sp.zeros(3,3), R_b]])*J_manipulator


J_geometric = J_b*T_A.row_join(J_eb)

J_g_controlled = J_geometric[:,0:4].row_join(J_geometric[:,6:9])
J_g_uncontrolled = J_geometric[:,4:6]

J_analytical = FK_centralised.jacobian(state_vector)

J_A_controlled = J_analytical[:,0:4].row_join(J_analytical[:,6:9])
J_A_uncontrolled = J_analytical[:,4:6]

J_Ac = sp.lambdify(state_vector, J_A_controlled)
J_Au = sp.lambdify(state_vector, J_A_uncontrolled)
print("Shape and dependencies of the controlled and uncontrolled Jacobians (rows, columns):")
sp.pprint(f"J_con = f({J_Ac.free_symbols})\t Shape = {J_Ac.shape}")
sp.pprint(f"J_unc = f({J_Au.free_symbols})\t Shape = {J_Au.shape}")

sp.pprint(f"T_A = f({T_A.free_symbols})")
sp.pprint(f"FK_centralised = f({FK_centralised.free_symbols})")
# Sample values for x, y, z, yaw, pitch, roll, q1, q2, q3
#J = J.subs([(x, 0), (y, 1.5), (z, -2.0), (yaw, sp.pi/4.), (pitch, 0.0), (roll, 0.0), (q1, -sp.pi/6), (q3, sp.pi/6)])

Centralised Jacobian shape: (6, 9)
Shape and dependencies of the controlled and uncontrolled Jacobians (rows, columns):
J_con = f({pitch, yaw, roll, q_2, q_3, q_1})	 Shape = (6, 7)
J_unc = f({pitch, yaw, roll, q_2, q_3, q_1})	 Shape = (6, 2)
T_A = f({pitch, yaw})
FK_centralised = f({z_b, y_b, pitch, roll, yaw, x_b, q_2, q_3, q_1})


### Inverse kinematics
From this, we follow the formulation of inverse kinematics through Closed Loop Inverse Kinematics (CLIK) from Arleo 2013 and Cataldi 2016. The formulation from Arleo is missing the inverse notation in the paper and the formulation of Cataldi involves a weighted pseudoinverse to bias the controller towards using the manipulator. We can investigate this later.

In [ ]:
print("Setting up dzeta/dt function")
# Instantaneous states
xb_inst = 0.0
yb_inst = 0.0
zb_inst = 0.0
yaw_inst = sp.pi/7
pitch_inst = 0.0
roll_inst = 0.0
q1_inst = -sp.pi/6
q2_inst = 0.0
q3_inst = sp.pi/6


# inputs
x_dot_ed = sp.Matrix([0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]) # desired end-effector velocity (set by controller)
x_ed = sp.Matrix([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # desired end-effector pose (set by controller)
K = sp.eye(6) # gain matrix (static)
sigma_dot = sp.Matrix([0.0, 0.0]) # uncontrolled joint velocities, pitch and roll rates (measured)

# Set up the CLIK differential equation
#J_pseudoinverse = J_controlled.T*sp.Inverse(J_controlled*J_controlled.T)
#controlled_states_ref_dot = J_pseudoinverse*T_A*(x_dot_ed+K*(x_ed - FK_centralised)) - J_pseudoinverse*J_uncontrolled*sigma_dot
#sp.pprint(f"State dependencies of controlled states reference velocities: {controlled_states_ref_dot.free_symbols}")


Setting up dzeta/dt function


KeyboardInterrupt: 

The initial idea was to derive symbolically the differential equation that is being reported (Arleo, 2013):
$$
\dot{\zeta}_{r}=J^{\dagger}_{\zeta}(\sigma_{b}, \zeta_{r})T_{A}(\Phi_{e,r})(\dot{x}_{e,d}+Ke)-J^{\dagger}_{\zeta}(\sigma_{b}, \zeta_{r})J_{\sigma}(\sigma_{b}, \zeta_{r})\dot{\sigma}_{b} \\
J^{\dagger}_{\zeta} = J_{\zeta}^{\top}(J_{\zeta}J_{\zeta}^{\top})^{-1}
$$
However this is impractical due to the complexity of the inversion operation. There are several other approaches that could supplement this:
- Given that the system cannot track both generalized coordinates and their derivates at the same time, get the measured values of the current generalized coordinates and use these as the input to the equation. Then pass the derivatives back as references.
- Employ a position inverse kinematics scheme to find $\zeta_{r}$ from $x_{e,d}$, the desired end-effector pose (note that $x_{e,r}$ denotes the end-effector pose computed on the basis of $\zeta_{r}$)

Let's investigate these options.

### Algorithm for option 1
We follow the formulation by Siciliano 2009, page 133, with the partitioning into controllable and uncontrollable parts as in Arleo 2013.
1. We set up the analytical Jacobian symbolically $J_{A}(\xi)$. This is programmed as a callable function returning the evaluated Jacobian matrix. We split the Jacobian into the controllable and uncontrollable Jacobians by columns.
2. We measure or estimate all states. The servo positions can be measured from the encoders, the drone orientation from the Kalman filter and the drone position from the distance sensor and OF sensors.
3. Evaluate $J_{A}$.
4. Calculate forward kinematics to find $x_{e}$.
5. Calculate error $e=x_{d}-x_{e}$, with $x_{d}$ the desired end-effector setpoint.
6. Multiply error by gain matrix $K$ and add to $\dot{x}_{d}$, the desired end-effector velocity: $\dot{x}_{d}+K(x_{d}-x_{e})$
7. Invert numerical Jacobian and multiply by the error term to obtain references for the states: $\dot{\zeta}=J_{a}^{-1}(\xi)$
8. Feed $\dot{\xi}$ as reference to the actuators.
9. Repeat.